# 外键
- 在MySQL中，表有两种引擎，一种是InnoDB，一种是myisam，如果使用的是InnoDB引擎，是支持外键约束的。外键的存在使得ORM框架在处理表关系的时候异常的强大
- 类定义为class ForeignKey(to, on_delete, **options).
    - 第一个参数是引用的是哪个模型，第二个参数是使用外键引用的模型数据被删除了后这个字段该如何处理，比如有CASCADE,SET_NULL等。
    - 当使用的是自己APP下的模型，直接引用名字即可，如果是想引用自己这个模型，还可以使用self。如果是引用外界APP的模型，使用appname.modelsname
- 外键是另一个模型的主键，拥有外键的称为子表（即引用者），被引用了主键的那张表称为主表（即被引用者）
- 示例如下：


In [1]:
#models文件中
from django.db import models

# Create your models here.
class Article(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=25)
    content = models.TextField()
    category = models.ForeignKey('Category', on_delete=models.CASCADE)
    #引用另外一个模块中的模型时，需要把模型名字加在模型的前面即: appname.modelsname
    dynasty = models.ForeignKey('anther_app.Dynasty', on_delete=models.CASCADE, null=True)

class Category(models.Model):
    name = models.CharField(max_length=50)


class Comment(models.Model):
    content = models.TextField()
    # 引用自身，可以使用self或者Comment,或者index_app.Comment
    original_comm = models.ForeignKey("self", on_delete=models.CASCADE)


ModuleNotFoundError: No module named 'django'

In [ ]:
# views.py文件中
from django.shortcuts import render
from .models import Category, Article
from django.http import HttpResponse

# Create your views here.
def index(request):
    category = Category(name="小说")
    category.save() #需要先保存外键，否则会报错
    article = Article(title='神话', author='taotao', content="解开我最神秘的等待，星星坠落时风儿在吹动")
    article.category = category
    article.save()

    get_article = Article.objects.first()
    print(get_article.category.name)  #直接通过属性的方式即可访问
    return HttpResponse("保存数据成功")

def delete(request):
    articles = Article.objects.all()
    articles.delete()
    return HttpResponse("删除操作成功")

# 外键的删除操作
- 如果一个模型使用了外键（即另一个模型的主键），那么在另一个模型的那条数据被删掉后，本模型对应的这条数据该进行什么样的操作，可以通过on_delete来指定。可指定的类型如下：
    - 1.CASCADE:级联操作，表示如果外键对应的那条数据被删除了，那么这条数据也会被删除
    - 2.PROTECT：受保护，表示被引用的那条数据不能被删除
    - 3.SET_DEFAULT:设置默认值，如果外键的那条数据被删除了，那么本条数据就将这个字段设置为默认值，如果设置这个选项，前提是要指定给这个字段一个默认值
    - 4.SET_NULL:设置为空，如果外键的那条数据被删除了，那么本条数据就将这个字段设置为空，前提是要指定这个字段可以为空
    - 5.SET():如果外键的那条数据被删除了，那么将会获取SET函数中的值来作为这个外键的值，SET函数可以接受一个可以调用的对象（比如函数或者方法），如果是可以调用的对象，那么会将这个对象调用后的结果作为值返回回去
    - 6.DO_NOTHING:不采取任何行为，一切全看数据库级别的约束
- 注意：以上这些选项只是django级别的，数据级别依旧是RESTRICT(和受保护类似的）

# 表关系
- 表之间的关系都是通过外键来进行关联的，而表之间的关系，无非就是三种关系：一对一，一对多，多对多

## 一对多：
- 应用场景：比如文章和作者之间的关系
- 实现方式：一对多还是多对一都是通过ForeignKey来实现的

### 知识点
#### 主表与子表的相互查询
-  主表查子表
    - 第一种办法：
        - 采用： 主表对象.引用者小写名字_set.方法
        - 示例：
                category = Category.objects.get(pk=2)
                articles = category.article_set.all()
        - article_set的解释：它是在通过外键生成一个子表的时候，django根据 子表的小写名+_set形式为被子表自动添加的属性。如果使用的方法是all(),返回的是一个querySet对象（是一个列表）
    - 第二种办法：
        - 采用：主表对象.自己定义的名字.方法
        - 自己定义的名字 是在子表的模型函数中使用ForeignKey函数时添加的，related_name="自己定义的名字"。这种方法和第一种只能取其一种，同时写入的话，就会报错
        - 示例：定义模型时：
                category = models.ForeignKey('Category',on_delete=models.CASCADE,related_name="new_article")
                category = Category.objects.get(pk=2)
                articles = category.new_article.all()
                
    
- 子表查主表
    - 先找到一个子表对象，然后通过 子表对象.外键关键字.查询的主表属性 即可查询。说白了就是通过外键跑到主表中去了，然后可以对主表的信息进行查找
    - 示例：
            magazine = Magazines.objects.get(pk=29)
            tags_result = magazine.tag.all()  #tag是那个外键
            for rel in tags_result:
                print(rel.tag_name)
    


#### 将已绑定的被引用者的属性值赋值给引用者属性的方法
- 第一种：
    - 采用：获取被引用者该外键，然后直接赋值
    - 示例：
            category = Category.objects.get(pk=2)
            article.category = category
- 第二种：
    - 采用:先通过一获取所有的多，然后将新对象加到多中
    - 示例：
            category = Category.objects.get(pk=2)
            articles = category.article_set.all()
            article = Article(title="",...)
            article.save()
            category.articles.add(article) #将category的属性赋值给article
            category.save()
    - 注意：article.save()一定要保存，而且注意save()是在给article的属性category赋值之前执行的，所以要求必须要将article字段category的null属性设置为True，否则在这里无法保存。它会产生一个死循环
        - 如果null属性值不为True，为False，则不能再使用save()保存，而应该在保存时添加参数：bulk=False，上述代码变为：
                category = Category.objects.get(pk=2)
                articles = category.article_set.all()
                article = Article(title="",...)
                category.articles.add(article, bulk=False) #将category的属性赋值给article
                category.save()
    - 所以推荐直接使用添加bulk=False参数，就不用管null属性是否为True或False



## 一对一：
- 应用场景：比如一个用户和一个用户信息表，在实际网站中，可能需要保存用户的许多信息，但是有些信息是不经常使用的。如果把所有信息都存放到一张表中可能会影响查询效率，因此可以把用户的一些不常用的信息存放到另一张表中我们叫做UserExtension。它们两者之间就是典型的一对一关系
- 实现方式：django专门为一对一提供了一个专门的Field叫做OneToOneField
- 外键数据的关键字在被引用者表中必须是唯一的，来保证一对一
- 同样也可以设置related_name参数，作用和上面一样

### 知识点
#### 主表与子表的相互查询
#### 注：使用表的小写名的前提是在表与表在关联的时候没有设置related_name
- 1.主表查子表中某个字段
    - 先得到一个主表的某一个对象，然后使用  对象.子表的小写名.访问的字段名  的方法即可访问
    - 示例：
            jj = User.objects.get(pk=1)
            获取jj的学校
            jj_school = jj.userextension.school
- 2.子表查主表中某个字段
    - 先得到一个子表的一个对象，然后使用  对象.主表的小写名.访问的字段名
    - 示例：
            jj_extension = UserExtension.objects.get(pk=1)
            获取jj的名字
            jj_name = jj_extension.user.username




## 多对多
- 应用场景：比如文章和标签的关系，一篇文章可以有多个标签，一个标签可以被多个文章所引用。
- 实现方式：django为这种多对多的实现提供了专门的Field，叫做ManyToManyField

### 知识点
#### 主表与子表的赋值
- 不能通过直接的方法将主表的主键（对于子表来说为外键）赋值给子表的外键，要通过add或者set方法
    - add方法可以一个一个的添加或者使用 *可迭代对象 方法
            magazine1.tag.add(tag1)
            magazine2.tag.add(*[tag1, tag2])
    - set方法,添加的各可迭代对象
            magazine2.tag.set([tag1, tag2])
                
#### 主表与子表的相互查询
- 主表查子表
    - 先找到一个主表对象，然后  主表对象.子表小写名_set.方法即可得到一个object对象，再object.字段名即可访问
    - 示例：
            tag = Tags.objects.get(pk=23)
            magazines_result = tag.magazines_set.all()
            for rel in magazines_result:
                print(rel.title)
                
- 子表查主表
    - 先找到一个子表对象，然后通过 子表对象.外键关键字.查询的主表属性 即可查询。说白了就是通过外键跑到主表中去了，然后可以对主表的信息进行查找
    - 示例：
            magazine = Magazines.objects.get(pk=29)
            tags_result = magazine.tag.all()  #tag是那个外键
            for rel in tags_result:
                print(rel.tag_name)


# 注：
- 凡是子表皆有  子表名_set属性，这是默认添加的，设置外键时都可以设置related_name="yourname"参数，这样"yourname"就会覆盖  子表名_set属性
- 一对一和一对多都可以通过直接赋值的方式把主表主键赋值给子表的外键，但多对多不行，只能通过add方法和set方法。赋值前一定要先保存主表
- 一对一和一对多如果通过add方法赋值时没有保存，可在add方法中添加参数bulk=False实现自动保存（推荐使用这种方法）。但是多对多不行，只能采用save方法

In [ ]:
from django.shortcuts import render
from .models import Category, Article
from django.http import HttpResponse

# Create your views here.
def index(request):
    category = Category(name="小说")
    category.save()  # 需要先保存外键，否则会报错
    article = Article(title='神话', author='taotao', content="解开我最神秘的等待，星星坠落时风儿在吹动")
    article.category = category
    article.save()

    get_article = Article.objects.first()
    print(get_article.category.name)  # 直接通过属性的方式即可访问
    return HttpResponse("保存数据成功")


def one_to_many(request):
    # 多对一，即多篇文章对应一种分类
    category = Category.objects.get(pk=2)
    article = Article(title='神话', author='taotao', content="解开我最神秘的等待，星星坠落时风儿在吹动")
    article.category = category
    # category.articles.
    article.save()

    article1 = Article(title='老人与海', author='haimingwei', content='他说风雨中，这点痛算什么，擦干泪不要问为什么')
    article1.category = category
    article1.save()

    category1 = Category(name="恋爱")
    category1.save()
    article2 = Article(title="最佳贴身男友", author='jiaojiao', content="你是爱我的，我也是。余生就让我来保护你把")
    article2.category = category1
    article2.save()

    return HttpResponse("构建多对一完成")



def get_result(request):
    # 获取某一分类下的所有文章： 被引用者实例.引用者小写名_set.方法
    # category = Category.objects.get(pk=2)
    # category1 = Category.objects.get(pk=7)
    # articles = category.article_set.all()
    # # article_set是被引用者被引用后，根据引用对象的小写+_set形成，它可以获取到哪些默型引用了该对象
    # # 用all得到是一个RelatedManager对象，querySet类型
    # print(articles, type(articles))
    # for article in articles:
    #     print(article)
    #
    # articles1 = category1.article_set.first()
    # print(articles1, type(articles1))


    # 如果不想使用被  引用者实例.引用者小写名.方法来获取某一分类下的所有文章
    # 引用者可以在引用时添加参数related_name = "自己设定的名字"，注意是在模型中设定
    # 就可以用   引用者实例.设定的名字.方法来获取, 这种方法和上一种方法只能取一种，否则要报错
    # articles = category.articles.all()
    # print(articles, type(articles))

    # 另外一种给引用者关联被引用者的方法
    # article3 = Article(title="恋爱了", author="xiaoxiao", content="爱是什么")
    # article3.save()  #请注意：此时article3没有category都可以保存成功，因为category设置了可以为空
    # category.articles.add(article3)

    # 如果category设置了不能为空，则上述方法就不管用了，会产生一个死循环,就应该采用如下方法
    # category.articles.add(article3, bulk=False)
    # category.save()

    # 主表查子表,通过  主表对象.子表名_set.访问的字段名  的方法找到所有的文章，然后用for循环打印出所访问的字段名
    # 子表查主表: 子表对象.主表的小写名.访问的字段名
    arti = Article.objects.get(pk=32)
    arti_category = arti.category.name #对象.主表的小写名.访问的字段名
    print(arti_category)

    return HttpResponse("获取结果完成")


def delete(request):
    articles = Article.objects.all()
    articles.delete()
    return HttpResponse("删除操作成功")




In [ ]:
from django.shortcuts import render
from .models import User, UserExtension
from django.http import HttpResponse

# Create your views here.
# 实现一对一
def one_to_one(request):
    jiaojiao = User(username="jiaojiao", password="123")
    jiaojiao.save()
    jj_extension = UserExtension(school="四川大学", city="成都")
    jj_extension.user = jiaojiao
    jj_extension.save()
    xiaoxiao = UserExtension(school="上海科技大学", city="上海")
    # xiaoxiao.user = jiaojiao  #此时就不能把jiaojiao的id赋值给另外一个对象，因为是一对一的关系
    xiaoxiao.save()

    # 通过主表查找子表的信息
    jj = User.objects.get(pk=1)
    print(jj.userextension.school)
    # 通过子表查找主表的信息
    jj_ext = UserExtension.objects.get(pk=1)
    print(jj_ext.user.username)

    return HttpResponse("一对一操作成功")


In [1]:
# 多对多举例
#models.py
from django.db import models


class Tags(models.Model):
    tag_name = models.CharField(max_length=100, null=True)

class Magazines(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    tag = models.ManyToManyField("Tags", null=True)
    
    
# views.py
from django.shortcuts import render
from .models import Tags, Magazines
from django.http import HttpResponse

# Create your views here.
# 实现一对一
def index(request):
    return HttpResponse("欢迎来到首页")

def many_to_many(request):
    #先对两个表输入数据
    tag1 = Tags(tag_name="最新杂志")
    tag2 = Tags(tag_name="最新节选")
    tag3 = Tags(tag_name="zuimeinvren")
    tag4 = Tags(tag_name="最火")
    tag3.save()
    tag4.save()
    tag1.save()
    tag2.save()
    magzine1 = Magazines.objects.create(title="最美女郎", content="这里有全球最美的女人")
    magzine1.save()
    magzine2 = Magazines.objects.create(title="最帅屌丝", content="屌丝这里应有尽有")
    magzine2.save()



    return HttpResponse("数据保存成功")


def exchange(request):
    # 找到第一个标签
    tag1 = Tags.objects.first()
    tag2 = Tags.objects.last()
    # 将第一个标签给同时给到杂志1和2
    magazine1 = Magazines.objects.first()
    magazine2 = Magazines.objects.last()
    # 不能通过直接赋值的方法把tag1的值赋值给magazine，即magzine.tag = tag1
    # magazine1.tag.add(tag1)  #给magazine1的属性tag添加值tag1
    magazine1.tag.add(tag2)  # 给magazine1的属性tag添加值tag2
    # 还可以通过set方法一次性给magazine1添加tag1和tag2，或者add的另一种方法
    # magazine2.tag.set([tag1, tag2])  #set方法是添加可迭代的对象
    # magazine2.tag.add(*[tag1, tag2])
    # magazine2.tag.add(tag1)

    return HttpResponse("操作数据成功")

def look_for(request):
    # 通过标签（主表）查找杂志（子表）,引用者即子表才有 表小写名_set属性
    # tag = Tags.objects.get(pk=23)
    # magazines_result = tag.magazines_set.all()
    # for rel in magazines_result:
    #     print(rel.title)

    # 通过杂志（子表）查找标签（主表），先找到一个子表对象，然后通过 子表对象.外键关键字.查询的主表属性 即可查询
    # 说白了就是通过外键跑到主表中去了，然后可以对主表的信息进行查找
    magazine = Magazines.objects.get(pk=29)
    tags_result = magazine.tag.all()
    for rel in tags_result:
        print(rel.tag_name)
    return HttpResponse("查找成功")

ModuleNotFoundError: No module named 'django'